In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy.io as sio
from keras.utils import np_utils

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, cohen_kappa_score
from sklearn.decomposition import PCA

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Reshape, Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

import cv2
from skimage.util import view_as_windows

In [2]:
from config.utils import *
from config.models import *

In [3]:
######### GLOBAL PARAMETERS #########
dataset = 'SA'
testSize = 0.84
windowSize = 25
K = 30 if dataset == 'IP' else 15
EPOCHS = 20
######### GLOBAL PARAMETERS #########

In [4]:
X, Y = loadData(dataset)

In [5]:
units, target_names = targets(dataset)

In [6]:
print(units)

16


In [7]:
X.shape

(512, 217, 204)

In [8]:
Y.shape

(512, 217)

In [9]:
X,pca = applyPCA(X,N=K)
ev=pca.explained_variance_ratio_
print('X.shape',X.shape)
print('y.shape',Y.shape)

X.shape (512, 217, 15)
y.shape (512, 217)


In [10]:
X , Y = createImageCubes(X,Y,windowSize=windowSize)

In [11]:
print('X.shape',X.shape)
print('y.shape',Y.shape)

X.shape (54129, 25, 25, 15)
y.shape (54129,)


In [12]:
X_Train , X_Test , Y_Train, Y_Test = train_test_split(X,Y,test_size=testSize,random_state=345)

In [13]:
Y_Train = np_utils.to_categorical(Y_Train)
Y_Train.shape

(8660, 16)

In [14]:
X_Train = X_Train.reshape(-1, windowSize, windowSize, K, 1)
X_Train.shape

(8660, 25, 25, 15, 1)

In [15]:
X_Test = X_Test.reshape(-1, windowSize, windowSize, K, 1)
X_Test.shape

(45469, 25, 25, 15, 1)

In [16]:
Y_Test = np_utils.to_categorical(Y_Test)
Y_Test.shape

(45469, 16)

In [17]:
X_Train.shape

(8660, 25, 25, 15, 1)

In [18]:
Model = make_3d_1(units, dataset)

In [19]:
Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 23, 23, 11, 8)     368       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 21, 21, 9, 16)     3472      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 10, 10, 4, 16)     0         
_________________________________________________________________
batch_normalization (BatchNo (None, 10, 10, 4, 16)     64        
_________________________________________________________________
reshape (Reshape)            (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 8, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

In [20]:
adam = Adam(lr=0.001, decay=1e-06)
Model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

filepath = "first" + dataset + ".hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_loss',patience=2)

callbacks_list = [checkpoint]

In [2]:
History = Model.fit(x=X_Train, y=Y_Train,epochs=EPOCHS, batch_size = 128, callbacks = callbacks_list)

In [22]:
Y_Pred = Model.predict(X_Test)
Y_Pred = np.argmax(Y_Pred, axis=1)

In [23]:
Report = (classification_report(np.argmax(Y_Test, axis=1), Y_Pred, target_names=target_names))
print(Report)

Matrix = (confusion_matrix(np.argmax(Y_Test, axis=1), Y_Pred))

                           precision    recall  f1-score   support

    Brocoli_green_weeds_1       1.00      1.00      1.00      1417
    Brocoli_green_weeds_2       1.00      1.00      1.00      2598
                   Fallow       1.00      0.98      0.99      1367
        Fallow_rough_plow       0.95      1.00      0.98       983
            Fallow_smooth       1.00      0.97      0.99      1852
                  Stubble       1.00      1.00      1.00      2779
                   Celery       0.98      1.00      0.99      2463
         Grapes_untrained       0.76      1.00      0.86      7885
     Soil_vinyard_develop       1.00      1.00      1.00      4324
Corn_senesced_green_weeds       1.00      0.99      1.00      2305
      Lettuce_romaine_4wk       1.00      0.99      0.99       777
      Lettuce_romaine_5wk       1.00      0.98      0.99      1376
      Lettuce_romaine_6wk       0.97      0.99      0.98       632
      Lettuce_romaine_7wk       0.99      1.00      1.00     

In [19]:
Model = make_3d_2(units, dataset)

In [20]:
Model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 23, 23, 9, 8)      512       
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 21, 21, 5, 16)     5776      
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 19, 19, 3, 16)     6928      
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 17, 17, 1, 16)     6928      
_________________________________________________________________
batch_normalization_1 (Batch (None, 17, 17, 1, 16)     64        
_________________________________________________________________
reshape_1 (Reshape)          (None, 17, 17, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 64)       

In [21]:
adam = Adam(lr=0.001, decay=1e-06)
Model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

filepath = "second" + dataset + ".hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_loss',patience=2)

callbacks_list = [checkpoint]

In [1]:
History = Model.fit(x=X_Train, y=Y_Train,epochs=EPOCHS, batch_size = 128, callbacks = callbacks_list)

In [23]:
Y_Pred = Model.predict(X_Test)
Y_Pred = np.argmax(Y_Pred, axis=1)

In [24]:
Report = (classification_report(np.argmax(Y_Test, axis=1), Y_Pred, target_names=target_names))
print(Report)
Matrix = (confusion_matrix(np.argmax(Y_Test, axis=1), Y_Pred))

                           precision    recall  f1-score   support

    Brocoli_green_weeds_1       1.00      1.00      1.00      1706
    Brocoli_green_weeds_2       0.97      1.00      0.98      3110
                   Fallow       1.00      0.67      0.80      1662
        Fallow_rough_plow       1.00      0.82      0.90      1171
            Fallow_smooth       0.75      1.00      0.85      2228
                  Stubble       1.00      1.00      1.00      3338
                   Celery       1.00      1.00      1.00      2984
         Grapes_untrained       0.98      0.87      0.92      9447
     Soil_vinyard_develop       1.00      1.00      1.00      5181
Corn_senesced_green_weeds       1.00      1.00      1.00      2752
      Lettuce_romaine_4wk       0.99      0.34      0.51       907
      Lettuce_romaine_5wk       1.00      0.96      0.98      1637
      Lettuce_romaine_6wk       0.53      0.99      0.69       773
      Lettuce_romaine_7wk       0.99      0.98      0.98     